基于MindSpore NLP的Roberta模型Prompt Tuning

原文链接：https://www.hiascend.com/forum/thread-02107166711681514014-1-1.html
本文档介绍了如何基于MindSpore NLP进行Roberta模型的Prompt Tuning，主要用于GLUE基准数据集的微调。本文提供了完整的代码示例以及详细的步骤说明，便于理解和复现实验。

配置环境

在运行此代码前，请确保MindSpore NLP库已经安装。本文档基于大模型平台运行，因此需要进行适当的环境配置，确保代码可以在相应的平台上运行。

模型与数据集加载

在本案例中，我们使用 roberta-large 模型并基于GLUE基准数据集进行Prompt Tuning。GLUE (General Language Understanding Evaluation) 是自然语言处理中的标准评估基准，包括多个子任务，如句子相似性匹配、自然语言推理等。Prompt Tuning是一种新的微调技术，通过插入虚拟的“提示”Token在模型的输入中，以微调较少的参数达到较好的性能。
————————————————

                            版权声明：本文为博主原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接和本声明。
                        
原文链接：https://blog.csdn.net/Kenji_Shinji/article/details/144395136

In [ ]:
import mindspore
from tqdm import tqdm
from mindnlp import evaluate
from mindnlp.dataset import load_dataset
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from mindnlp.core.optim import AdamW
from mindnlp.transformers.optimization import get_linear_schedule_with_warmup
from mindnlp.peft import (
    get_peft_model,
    PeftType,
    PromptTuningConfig,
)

01

定义训练参数 

首先，定义模型名称、数据集任务名称、Prompt Tuning类型、训练轮数等基本参数。

In [ ]:
batch_size = 32
model_name_or_path = "roberta-large"
task = "mrpc"
peft_type = PeftType.PROMPT_TUNING
num_epochs = 20

02

配置Prompt Tuning

在Prompt Tuning的配置中，选择任务类型为"SEQ_CLS"（序列分类任务），并定义虚拟Token的数量。虚拟Token即为插入模型输入中的“提示”Token，通过这些Token的微调，使得模型能够更好地完成下游任务。

In [ ]:
peft_config = PromptTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=10)
lr = 1e-3

03

加载Tokenizer 

根据模型类型选择padding的侧边，如果模型为GPT、OPT或BLOOM类模型，则从序列左侧填充（padding），否则从序列右侧填充。

In [ ]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"
 
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

04

加载数据集 

通过MindSpore NLP加载GLUE数据集，并打印样本以便确认数据格式。在此示例中，我们使用GLUE的MRPC（Microsoft Research Paraphrase Corpus）任务，该任务用于句子匹配，即判断两个句子是否表达相同的意思。

In [ ]:
datasets = load_dataset("glue", task)
print(next(datasets['train'].create_dict_iterator()))

05

数据预处理  

为了适配MindSpore NLP的数据处理流程，我们定义了一个映射函数 MapFunc，用于将句子转换为 input_ids 和 attention_mask，并对数据进行padding处理。

In [ ]:
from mindnlp.dataset import BaseMapFunction
 
class MapFunc(BaseMapFunction):
    def __call__(self, sentence1, sentence2, label, idx):
        outputs = tokenizer(sentence1, sentence2, truncation=True, max_length=None)
        return outputs['input_ids'], outputs['attention_mask'], label
 
def get_dataset(dataset, tokenizer):
    input_colums=['sentence1', 'sentence2', 'label', 'idx']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})
    return dataset
 
train_dataset = get_dataset(datasets['train'], tokenizer)
eval_dataset = get_dataset(datasets['validation'], tokenizer)

06

设置评估指标 

我们使用 evaluate 模块加载评估指标（accuracy 和 F1-score）来评估模型的性能。

In [ ]:
metric = evaluate.load("./glue.py", task)

07

加载模型并配置Prompt Tuning 

加载 roberta-large 模型，并根据配置进行Prompt Tuning。可以看到，微调的参数量仅为总参数量的0.3%左右，节省了大量计算资源。

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

模型微调（Prompt Tuning）

在Prompt Tuning中，训练过程中仅微调部分参数（主要是虚拟Token相关的参数），相比于传统微调而言，大大减少了需要调整的参数量，使得模型能够高效适应下游任务。

01

优化器与学习率调整 

使用 AdamW 优化器，并设置线性学习率调整策略。

In [ ]:
optimizer = AdamW(params=model.parameters(), lr=lr)
 
# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataset) * num_epochs),
    num_training_steps=(len(train_dataset) * num_epochs),
)

02

训练数据集 

训练步骤如下：

1、构建正向计算函数 forward_fn。

2、定义梯度计算函数 grad_fn。

3、定义每一步的训练逻辑 train_step。

4、遍历数据集进行训练和评估，在每个 epoch 结束时，计算评估指标。

In [ ]:
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss
 
grad_fn = mindspore.value_and_grad(forward_fn, None, tuple(model.parameters()))
 
def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer.step(grads)
    return loss
 
for epoch in range(num_epochs):
    model.set_train()
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        loss = train_step(**batch)
        lr_scheduler.step()
 
    model.set_train(False)
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        outputs = model(**batch)
        predictions = outputs.logits.argmax(axis=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )
 
    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

总结

本案例通过Prompt Tuning技术，在Roberta模型上进行了微调以适应GLUE数据集任务。通过控制微调参数量，Prompt Tuning展示了较强的高效性。